In [5]:
%run Model_And_Optimization.ipynb


import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

X, y = make_classification(n_samples=1000, n_features=10, random_state=42)

# Define models and parameter grids
models = [LogisticRegression(), RandomForestClassifier(), GradientBoostingClassifier()]

param_grids = [
    {'C': [0.1, 1, 10]},  # Parameter grid for LogisticRegression
    {'max_depth': [None, 5, 10], 'n_estimators': [50, 100, 150]},  # Parameter grid for RandomForestClassifier
    {'learning_rate': [0.1, 0.01, 0.001], 'n_estimators': [50, 100, 150]}  # Parameter grid for GradientBoostingClassifier
]

# Example: Use the ModelOptimization class to optimize models with roc_auc scoring
scoring = 'accuracy'
model_comparison = ModelOptimization(models, param_grids, scoring, cv=5, top_n=3)
model_comparison.fit(X, y)

# Make predictions using the best models
X_test, y_test = make_classification(n_samples=100, n_features=10, random_state=42)
predictions = model_comparison.predict(X_test)

# Print the predictions for each model
for model_name, y_pred in predictions.items():
    print(f"Predictions for {model_name}:\n{y_pred}\n")
    

Fitting model 1/3: LogisticRegression
Fitting model 2/3: RandomForestClassifier
Fitting model 3/3: GradientBoostingClassifier

Model ranking based on scores:
Rank 1: RandomForestClassifier - Score: 1.0000
Rank 2: GradientBoostingClassifier - Score: 0.9770
Rank 3: LogisticRegression - Score: 0.8590



Optimizing hyperparameters for model RandomForestClassifier
Best score for RandomForestClassifier: 0.9130
Params: {'max_depth': 10, 'n_estimators': 50}


Optimizing hyperparameters for model GradientBoostingClassifier
Best score for GradientBoostingClassifier: 0.9070
Params: {'learning_rate': 0.01, 'n_estimators': 150}


Optimizing hyperparameters for model LogisticRegression
Best score for LogisticRegression: 0.8580
Params: {'C': 0.1}


Predictions for RandomForestClassifier:
[1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 0 1 0 1 1 1 1 0 1 0 1 0 1 1 1 1 1 0 1 1 0 0
 1 0 0 0 0 1 1 1 0 1 0 0 1 0 0 1 1 1 0 0 1 1 0 1 1 0 0 1 1 1 1 1 0 1 0 0 1
 1 1 0 1 0 1 0 1 0 0 1 1 1 1 0 0 0 1 1 1 1 1 1 0 0 1]

Predictions